In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
!nvidia-smi

Sun Nov 26 16:50:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
%%writefile lab2_ex2.cu
#include <stdio.h>
#include <sys/time.h>

#define TPB 16
#define DataType double
#define DOUBLE_MIN -5
#define DOUBLE_MAX 5

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error: %s, line: %d\n", cudaGetErrorString(err), __LINE__); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

// Compute C = A * B
__global__ void gemm(DataType *A, DataType *B, DataType *C, int numARows,
                    int numAColumns, int numBRows, int numBColumns){
    //@@ Insert code to implement matrix multiplication here
    int index_x = blockIdx.x * blockDim.x + threadIdx.x;
    int index_y = blockIdx.y * blockDim.y + threadIdx.y;

    if (index_y < numARows && index_x < numBColumns)
    {
        DataType sum = 0;
        for (int k = 0; k < numAColumns; k++){
            sum += A[index_y * numAColumns + k] * B[k * numBColumns + index_x];
        }
        C[index_y * numBColumns + index_x] = sum;
    }
}

//@@ Insert code to implement timer start
void timerStart(struct timeval *start) {
    gettimeofday(start, NULL);
}

//@@ Insert code to implement timer stop
double timerStop(struct timeval *start) {
    struct timeval end;
    gettimeofday(&end, NULL);
    double time = (end.tv_sec - start->tv_sec) * 1000.0;
    time += (end.tv_usec - start->tv_usec) / 1000.0;
    return time;
}

double randDouble(double min, double max) {
    double scale = rand() / (double)RAND_MAX;
    return min + scale * (max-min);
}

int main(int argc, char **argv) {
    DataType *hostA; // The A matrix
    DataType *hostB; // The B matrix
    DataType *hostC; // The output C matrix
    DataType *resultRef; // The reference result
    DataType *deviceA;
    DataType *deviceB;
    DataType *deviceC;
    int numARows;    // number of rows in the matrix A
    int numAColumns; // number of columns in the matrix A
    int numBRows;    // number of rows in the matrix B
    int numBColumns; // number of columns in the matrix B
    int numCRows;
    int numCColumns;

    struct timeval copyToDevice, copyFromDevice, kernelExecution;
    double copyToDeviceTime, copyFromDeviceTime, kernelExecutionTime;

    //@@ Insert code below to read in numARows, numAColumns, numBColumns from args
    if (argc >= 4)
    {
        numARows = atoi(argv[1]);
        numAColumns = atoi(argv[2]);
        numBRows = numAColumns;
        numBColumns = atoi(argv[3]);
        numCRows = numARows;
        numCColumns = numBColumns;
    }
    printf("Input matrix dim (%d x %d) (%d x %d) (%d x %d)\n", numARows, numAColumns, numBRows, numBColumns, numCRows, numCColumns);

    //@@ Insert code below to allocate Host memory for input and output
    hostA = (DataType *)malloc(numARows * numAColumns * sizeof(DataType));
    hostB = (DataType *)malloc(numBRows * numBColumns * sizeof(DataType));
    hostC = (DataType *)malloc(numCRows * numCColumns * sizeof(DataType));
    resultRef = (DataType *)malloc(numCRows * numCColumns * sizeof(DataType));

    //@@ Insert code below to initialize hostA and hostB to random numbers, and create reference result in CPU
    //printf("\n\nhostA:\n");
    for(int i = 0; i < numARows; i++){
        for(int j = 0; j < numAColumns; j++){
            hostA[i * numAColumns + j] = randDouble(DOUBLE_MIN, DOUBLE_MAX);
            //printf("%.3f ", hostA[i * numAColumns + j]);
        }
        //printf("\n");
    }

    //printf("\n\nhostB:\n");
    for(int i = 0; i < numBRows; i++){
        for(int j = 0; j < numBColumns; j++){
            hostB[i * numBColumns + j] = randDouble(DOUBLE_MIN, DOUBLE_MAX);
            //printf("%.3f ", hostB[i * numBColumns + j]);
        }
        //printf("\n");
    }

    //printf("\n\nresultRef:\n");
    for(int i = 0; i < numCRows; i++){
        for(int j = 0; j < numCColumns; j++){
            resultRef[i * numCColumns + j] = 0;

            for (int k = 0; k < numAColumns; k++) {
                resultRef[i * numCColumns + j] += hostA[i * numAColumns + k] * hostB[k * numBColumns + j];
            }
            //printf("%.3f ", resultRef[i * numCColumns + j]);
        }
        //printf("\n");
    }


    //@@ Insert code below to allocate GPU memory here
    CUDA_CHECK(cudaMalloc((void **)&deviceA, numARows * numAColumns * sizeof(DataType)));
    CUDA_CHECK(cudaMalloc((void **)&deviceB, numBRows * numBColumns * sizeof(DataType)));
    CUDA_CHECK(cudaMalloc((void **)&deviceC, numCRows * numCColumns * sizeof(DataType)));


    //@@ Insert code to below to Copy memory to the GPU here
    timerStart(&copyToDevice);
    CUDA_CHECK(cudaMemcpy(deviceA, hostA, numARows * numAColumns * sizeof(DataType), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(deviceB, hostB, numBRows * numBColumns * sizeof(DataType), cudaMemcpyHostToDevice));
    copyToDeviceTime = timerStop(&copyToDevice);

    //@@ Initialize the grid and block dimensions here
    dim3 DimGrid((numCColumns + TPB - 1) / TPB, (numCRows + TPB - 1) / TPB, 1);
    dim3 DimBlock(TPB, TPB, 1);

    //@@ Launch the GPU Kernel here
    timerStart(&kernelExecution);
    gemm<<<DimGrid, DimBlock>>>(deviceA, deviceB, deviceC, numARows, numAColumns, numBRows, numCColumns);
    cudaDeviceSynchronize();
    kernelExecutionTime = timerStop(&kernelExecution);

    //@@ Copy the GPU memory back to the CPU here
    timerStart(&copyFromDevice);
    CUDA_CHECK(cudaMemcpy(hostC, deviceC, numCRows * numCColumns * sizeof(DataType), cudaMemcpyDeviceToHost));
    copyFromDeviceTime = timerStop(&copyFromDevice);

    //@@ Insert code below to compare the output with the reference
    double diff = 0.0;
    //printf("\n\nhostC:\n");
    for(int i = 0; i < numCRows; i++){
        for(int j = 0; j < numCColumns; j++) {
            diff += abs(hostC[i * numCColumns + j] - resultRef[i * numCColumns + j]);
            //printf("%.3f ", hostC[i * numCColumns + j]);
        }
        //printf("\n");
    }

    printf("Average difference: %f\n\n", diff/(double)(numCRows * numCColumns));

    printf("\nCopy to Device Time: %f ms\n", copyToDeviceTime);
    printf("Kernel Execution Time: %f ms\n", kernelExecutionTime);
    printf("Copy from Device Time: %f ms\n", copyFromDeviceTime);

    //@@ Free the GPU memory here
    cudaFree((void *) deviceA);
    cudaFree((void *) deviceB);
    cudaFree((void *) deviceC);

    //@@ Free the CPU memory here
    free(hostA);
    free(hostB);
    free(hostC);

    return 0;
}





Overwriting lab2_ex2.cu


In [5]:
!nvcc lab2_ex2.cu
!ls
!./a.out 128 128 128

a.out  lab2_ex2.cu  sample_data
Input matrix dim (128 x 128) (128 x 128) (128 x 128)
Average difference: 0.000000


Copy to Device Time: 1.259000 ms
Kernel Execution Time: 0.092000 ms
Copy from Device Time: 0.129000 ms


In [6]:
!nvprof ./a.out 128 128 128

Input matrix dim (128 x 128) (128 x 128) (128 x 128)
==1198== NVPROF is profiling process 1198, command: ./a.out 128 128 128
Average difference: 0.000000


Copy to Device Time: 0.116000 ms
Kernel Execution Time: 0.098000 ms
Copy from Device Time: 0.134000 ms
==1198== Profiling application: ./a.out 128 128 128
==1198== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   60.29%  64.575us         1  64.575us  64.575us  64.575us  gemm(double*, double*, double*, int, int, int, int)
                   28.08%  30.079us         2  15.039us  13.919us  16.160us  [CUDA memcpy HtoD]
                   11.62%  12.448us         1  12.448us  12.448us  12.448us  [CUDA memcpy DtoH]
      API calls:   99.24%  224.83ms         3  74.944ms  2.8230us  224.82ms  cudaMalloc
                    0.48%  1.0955ms         1  1.0955ms  1.0955ms  1.0955ms  cuDeviceGetPCIBusId
                    0.11%  244.25us         3  81.416us  54.616us  133.46u

In [7]:
!ncu ./a.out 128 128 128

Input matrix dim (128 x 128) (128 x 128) (128 x 128)
==PROF== Connected to process 1230 (/content/a.out)
==PROF== Profiling "gemm" - 0: 0%....50%....100% - 8 passes
Average difference: 0.000000


Copy to Device Time: 0.114000 ms
Kernel Execution Time: 400.403000 ms
Copy from Device Time: 0.158000 ms
==PROF== Disconnected from process 1230
[1230] a.out@127.0.0.1
  gemm(double *, double *, double *, int, int, int, int), 2023-Nov-26 20:21:26, Context 1, Stream 7
    Section: GPU Speed Of Light Throughput
    ---------------------------------------------------------------------- --------------- ------------------------------
    DRAM Frequency                                                           cycle/nsecond                           4.97
    SM Frequency                                                             cycle/usecond                         581.55
    Elapsed Cycles                                                                   cycle                         36,947
    M

In [8]:
!ncu ./a.out 511 1023 4094

Input matrix dim (511 x 1023) (1023 x 4094) (511 x 4094)
==PROF== Connected to process 1323 (/content/a.out)
==PROF== Profiling "gemm" - 0: 0%....50%....100% - 8 passes
Average difference: 0.000000


Copy to Device Time: 8.724000 ms
Kernel Execution Time: 686.054000 ms
Copy from Device Time: 13.442000 ms
==PROF== Disconnected from process 1323
[1323] a.out@127.0.0.1
  gemm(double *, double *, double *, int, int, int, int), 2023-Nov-26 20:22:21, Context 1, Stream 7
    Section: GPU Speed Of Light Throughput
    ---------------------------------------------------------------------- --------------- ------------------------------
    DRAM Frequency                                                           cycle/nsecond                           5.00
    SM Frequency                                                             cycle/usecond                         585.01
    Elapsed Cycles                                                                   cycle                     27,876,330
